## 0. Download dataset

In [5]:
!gdown --id 1kc6XNqHZJg27KeBuoAoYj70_1rT92191

/opt/anaconda3/lib/python3.12/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1kc6XNqHZJg27KeBuoAoYj70_1rT92191
From (redirected): https://drive.google.com/uc?id=1kc6XNqHZJg27KeBuoAoYj70_1rT92191&confirm=t&uuid=d6fab801-95c3-400b-81ff-4b17acea5110
To: /Users/hunghehe2205/ML_Learning/vqa_coco_dataset.zip
100%|████████████████████████████████████████| 196M/196M [00:18<00:00, 10.6MB/s]
